In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sentencepiece as spm
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

import re
import os
import random
import math

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

## Step 1. 데이터 다운로드

준비하기 단계에서 심볼릭 링크를 생성했다면 아래 파일이 ChatbotData .csv라는 이름으로 저장되어 있을거예요. csv 파일을 읽는 데에는 pandas 라이브러리가 적합합니다. 읽어 온 데이터의 질문과 답변을 각각 questions, answers 변수에 나눠서 저장하세요! [songys/Chatbot_data](https://github.com/songys/Chatbot_data)

In [2]:
chatbot_data = pd.read_csv('~/aiffel/transformer_chatbot/data/ChatbotData.csv')

In [3]:
chatbot_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [4]:
len(chatbot_data)

11823

In [5]:
questions = chatbot_data['Q']
answers =chatbot_data['A']

In [6]:
questions

0                         12시 땡!
1                    1지망 학교 떨어졌어
2                   3박4일 놀러가고 싶다
3                3박4일 정도 놀러가고 싶다
4                        PPL 심하네
                  ...           
11818             훔쳐보는 것도 눈치 보임.
11819             훔쳐보는 것도 눈치 보임.
11820                흑기사 해주는 짝남.
11821    힘든 연애 좋은 연애라는게 무슨 차이일까?
11822                 힘들어서 결혼할까봐
Name: Q, Length: 11823, dtype: object

In [7]:
answers

0                      하루가 또 가네요.
1                       위로해 드립니다.
2                     여행은 언제나 좋죠.
3                     여행은 언제나 좋죠.
4                      눈살이 찌푸려지죠.
                   ...           
11818          티가 나니까 눈치가 보이는 거죠!
11819               훔쳐보는 거 티나나봐요.
11820                      설렜겠어요.
11821    잘 헤어질 수 있는 사이 여부인 거 같아요.
11822          도피성 결혼은 하지 않길 바라요.
Name: A, Length: 11823, dtype: object

## Step 2. 데이터 정제
아래 조건을 만족하는 preprocess_sentence() 함수를 구현하세요.

1. 영문자의 경우, 모두 소문자로 변환합니다.
2. 영문자와 한글, 숫자, 그리고 주요 특수문자를 제외하곤 정규식을 활용하여 모두 제거합니다.

문장부호 양옆에 공백을 추가하는 등 이전과 다르게 생략된 기능들은 우리가 사용할 토크나이저가 지원하기 때문에 굳이 구현하지 않아도 괜찮습니다!

In [8]:
def preprocess_sentence(sentence):
    # 대문자를 소문자로 변환
    sentence = sentence.lower()
    # 두 개 이상의 공백을 하나로 변경
    sentence = re.sub(r'[" "]+', " ", sentence)
    # 영문자와 한글, 숫자, 그리고 주요 특수문자를 제외하곤 정규식을 활용하여 모두 제거합니다.
    sentence = re.sub(r"[^a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣?.!,]+", " ", sentence)
    # 문자열 양끝 공백 제거
    sentence = sentence.strip()
    return sentence

## Step 3. 데이터 토큰화
토큰화에는 KoNLPy의 mecab 클래스를 사용합니다.

아래 조건을 만족하는 build_corpus() 함수를 구현하세요!

1. 소스 문장 데이터와 타겟 문장 데이터를 입력으로 받습니다.
2. 데이터를 앞서 정의한 preprocess_sentence() 함수로 정제하고, 토큰화합니다.
3. 토큰화는 전달받은 토크나이즈 함수를 사용합니다. 이번엔 mecab.morphs 함수를 전달하시면 됩니다.
4. 토큰의 개수가 일정 길이 이상인 문장은 데이터에서 제외합니다.
5. 중복되는 문장은 데이터에서 제외합니다. 소스 : 타겟 쌍을 비교하지 않고 소스는 소스대로 타겟은 타겟대로 검사합니다. 중복 쌍이 흐트러지지 않도록 유의하세요!
구현한 함수를 활용하여 questions 와 answers 를 각각 que_corpus , ans_corpus 에 토큰화하여 저장합니다.

In [9]:
from konlpy.tag import Mecab

def build_corpus(questions, answers, max_len=100000):
    questions = list(map(preprocess_sentence, questions))
    answers = list(map(preprocess_sentence, answers))
    
    mecab = Mecab()
    que_corpus = [" ".join(mecab.morphs(q)) for q in questions]
    ans_corpus = [" ".join(mecab.morphs(a)) for a in answers]
    
    que_set = set()
    ans_set = set()
    new_que_corpus = []
    new_ans_corpus = []
    
    for que, ans in zip(que_corpus, ans_corpus):
        if (len(que.split()) > max_len) or (len(ans.split()) > max_len):
            continue
        if (que not in que_set) and (ans not in ans_set):
            new_que_corpus.append(que)
            new_ans_corpus.append(ans)
        que_set.add(que)
        ans_set.add(ans)
    
    return new_que_corpus, new_ans_corpus
    
    

In [10]:
que_corpus, ans_corpus = build_corpus(questions, answers)

In [11]:
que_corpus_check = np.array(list(map(lambda s: len(s.split()), que_corpus))) 
ans_corpus_check = np.array(list(map(lambda s: len(s.split()), ans_corpus))) 

In [12]:
que_corpus_check.mean(), que_corpus_check.std(), que_corpus_check.max(), que_corpus_check.min()

(7.459020686043467, 3.751139714643547, 32, 1)

In [13]:
ans_corpus_check.mean(), ans_corpus_check.std(), ans_corpus_check.max(), ans_corpus_check.min()

(8.697433883215501, 3.597476816886744, 40, 1)

In [14]:
max_len = 20
que_corpus, ans_corpus = build_corpus(questions, answers, max_len)

In [15]:
len(que_corpus), len(ans_corpus)

(7528, 7528)

In [16]:
que_corpus[10], ans_corpus[10]

('가난 한 자 의 설움', '돈 은 다시 들어올 거 예요 .')

## Step 4. Augmentation

우리에게 주어진 데이터는 1만 개가량으로 적은 편에 속합니다. 이럴 때에 사용할 수 있는 테크닉을 배웠으니 활용해 봐야겠죠? Lexical Substitution을 실제로 적용해 보도록 하겠습니다.

아래 링크를 참고하여 한국어로 사전 훈련된 Embedding 모델을 다운로드합니다. Korean (w) 가 Word2Vec으로 학습한 모델이며 용량도 적당하므로 사이트에서 Korean (w)를 찾아 다운로드하고, ko.bin 파일을 얻으세요!

[Kyubyong/wordvectors](https://github.com/Kyubyong/wordvectors)

다운로드한 모델을 활용해 데이터를 Augmentation 하세요! 앞서 정의한 lexical_sub() 함수를 참고하면 도움이 많이 될 겁니다.

Augmentation된 que_corpus 와 원본 ans_corpus 가 병렬을 이루도록, 이후엔 반대로 원본 que_corpus 와 Augmentation된 ans_corpus 가 병렬을 이루도록 하여 전체 데이터가 원래의 3배가량으로 늘어나도록 합니다.



In [17]:
# 한국어 word2vec이 최신버전에서 Can't get attribute 'Vocab' on <module 'gensim.models.word2vec' 오류가 있음.
!pip install --upgrade gensim==3.8.3

In [18]:
import gensim
# print(gensim.__version__)
wv = gensim.models.Word2Vec.load('~/aiffel/transformer_chatbot/data/ko.bin')

In [19]:
wv.most_similar("왕")

/tmp/ipykernel_821/2855753700.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  wv.most_similar("왕")


[('왕인', 0.7048283219337463),
 ('선왕', 0.6905209422111511),
 ('왕의', 0.6839720010757446),
 ('황제', 0.6579594612121582),
 ('대왕', 0.6463155150413513),
 ('성왕', 0.6421108245849609),
 ('문공', 0.6363499164581299),
 ('왕자', 0.6358417272567749),
 ('왕과', 0.6349694132804871),
 ('혜공', 0.6336162090301514)]

In [20]:
def lexical_sub(sample_sentence, wv):
    sample_tokens = sample_sentence.split()
    selected_tok = random.choice(sample_tokens)
    result = ""
    for tok in sample_tokens:
        if tok is selected_tok and tok not in "?.!,":
            try:
                result += wv.most_similar(tok)[0][0] + " "
            except:
                result += tok + " "

        else:
            result += tok + " "
    return result.strip()

In [21]:
idx_to_check = 10
que_corpus[idx_to_check], lexical_sub(que_corpus[idx_to_check], wv)

/tmp/ipykernel_821/3559501181.py:8: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  result += wv.most_similar(tok)[0][0] + " "


('가난 한 자 의 설움', '가난 한두 자 의 설움')

In [22]:
wv.most_similar("가난") # 헉......

/tmp/ipykernel_821/3270535881.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  wv.most_similar("가난") # 헉......


[('부유', 0.7616064548492432),
 ('유복', 0.6900136470794678),
 ('평범', 0.6759896278381348),
 ('궁핍', 0.6658532619476318),
 ('불우', 0.6463127136230469),
 ('곤궁', 0.6358442902565002),
 ('풍족', 0.6316250562667847),
 ('방탕', 0.6238061785697937),
 ('비참', 0.6231825947761536),
 ('검소', 0.6068432331085205)]

In [23]:
aug_ans_corpus = list(map(lambda s: lexical_sub(s, wv), ans_corpus))
aug_que_corpus = list(map(lambda s: lexical_sub(s, wv), que_corpus))

/tmp/ipykernel_821/3559501181.py:8: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  result += wv.most_similar(tok)[0][0] + " "


In [24]:
# data_pair0 = (que_corpus, ans_corpus)
# data_pair1 = (aug_que_corpus, ans_corpus)
# data_pair2 = (que_corpus, aug_ans_corpus)

## Step 5. 데이터 벡터화

타겟 데이터인 ans_corpus 에 <start> 토큰과 <end> 토큰이 추가되지 않은 상태이니 이를 먼저 해결한 후 벡터화를 진행합니다. 우리가 구축한 ans_corpus 는 list 형태이기 때문에 아주 쉽게 이를 해결할 수 있답니다!
    
```
sample_data = ["12", "시", "땡", "!"]
print(["<start>"] + sample_data + ["<end>"])
```
    
1. 위 소스를 참고하여 타겟 데이터 전체에 <start> 토큰과 <end> 토큰을 추가해 주세요!

챗봇 훈련 데이터의 가장 큰 특징 중 하나라고 하자면 바로 소스 데이터와 타겟 데이터가 같은 언어를 사용한다는 것이겠죠. 앞서 배운 것처럼 이는 Embedding 층을 공유했을 때 많은 이점을 얻을 수 있습니다.

2. 특수 토큰을 더함으로써 ans_corpus 또한 완성이 되었으니, que_corpus 와 결합하여 전체 데이터에 대한 단어 사전을 구축하고 벡터화하여 enc_train 과 dec_train 을 얻으세요!


In [25]:
ans_corpus = list(map(lambda s: "<start> " + s + " <end>", ans_corpus))
aug_ans_corpus = list(map(lambda s: "<start> " + s + " <end>", aug_ans_corpus))

In [26]:
# 데이터 증강
enc_corpus = que_corpus + aug_que_corpus + que_corpus
dec_corpus = ans_corpus + ans_corpus + aug_ans_corpus

In [27]:
len(enc_corpus), len(dec_corpus)

(22584, 22584)

In [28]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(enc_corpus + dec_corpus)
len(tokenizer.word_index)

7082

In [29]:
VOCAB_SIZE = 6000 # 거의 대부분 사용
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<UNK>", filters="")
tokenizer.fit_on_texts(enc_corpus + dec_corpus)

In [30]:
from keras.preprocessing.sequence import pad_sequences
enc_train = tokenizer.texts_to_sequences(enc_corpus)
enc_train = pad_sequences(enc_train, padding='post', maxlen=max_len)

In [31]:
dec_train = tokenizer.texts_to_sequences(dec_corpus)
dec_train = pad_sequences(dec_train, padding='post', maxlen=max_len+1)

In [32]:
enc_corpus[0], dec_corpus[0], enc_train[0], dec_train[0]

('시 땡 !',
 '<start> 하루 가 또 가 네요 . <end>',
 array([ 212, 3530,  102,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32),
 array([  3, 301,   9, 133,   9,  42,   2,   4,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0], dtype=int32))

## Step 6. 훈련하기

앞서 번역 모델을 훈련하며 정의한 Transformer 를 그대로 사용하시면 됩니다! 대신 데이터의 크기가 작으니 하이퍼파라미터를 튜닝해야 과적합을 피할 수 있습니다. 모델을 훈련하고 아래 예문에 대한 답변을 생성하세요! 가장 멋진 답변과 모델의 하이퍼파라미터를 제출하시면 됩니다.

```
# 예문
1. 지루하다, 놀러가고 싶어.
2. 오늘 일찍 일어났더니 피곤하다.
3. 간만에 여자친구랑 데이트 하기로 했어.
4. 집에 있는다는 소리야.

---

# 제출

Translations
> 1. 잠깐 쉬 어도 돼요 . <end>
> 2. 맛난 거 드세요 . <end>
> 3. 떨리 겠 죠 . <end>
> 4. 좋 아 하 면 그럴 수 있 어요 . <end>

Hyperparameters
> n_layers: 1
> d_model: 368
> n_heads: 8
> d_ff: 1024
> dropout: 0.2

Training Parameters
> Warmup Steps: 1000
> Batch Size: 64
> Epoch At: 10
```

### 모델 구현

In [33]:
# Positional Encoding 구현
def positional_encoding(pos, d_model):
    sinusoid_table = np.array([[pos_i / np.power(10000, 2 * (d_j // 2) / np.float32(d_model)) for d_j in range(d_model)] for pos_i in range(pos)])
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])

    return sinusoid_table

In [34]:
# Mask  생성하기
def generate_padding_mask(seq):
    # 시퀀스에서 각 위치의 값이 0인지 확인
    mask = tf.cast(tf.math.equal(seq, 0), tf.float32)
    # 차원 추가하여 마스크의 형태를 모델에 맞게 조정
    # 예: (batch_size, 1, 1, sequence_length) 
    return mask[:, tf.newaxis, tf.newaxis, :]

def generate_lookahead_mask(size):
    # mask 부분에 대해서 구하려면 1에서 빼줌
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_enc_mask = generate_padding_mask(src)

    # dec_mask is masked attention
    lookahead_mask = generate_lookahead_mask(tgt.shape[1])
    padding_mask = generate_padding_mask(tgt)
    dec_mask = tf.maximum(lookahead_mask, padding_mask)

    return enc_mask, dec_enc_mask, dec_mask

In [35]:
# Multi Head Attention 구현
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        
        self.depth = d_model // self.num_heads
        
        # for multi head split
        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)
    
        # for multi head combine
        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        # scaled_QK = Q K^t / sqrt(d_k) 
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)
        scaled_QK = QK / tf.math.sqrt(d_k) # shape: (seq, seq)

        if mask is not None:
            # softmax를 취하기 때문에 -무한대를 주면 해당 값이 0이 됨.
            scaled_QK += (mask * -1e9)

        attentions = tf.nn.softmax(scaled_QK, axis=-1)
        out = tf.matmul(attentions, V)
        
        return out, attentions
        
    def split_heads(self, x):
        # (batch_size, seq, d_model) -> (batch_size, seq, num_heads, depth)
        split_x = tf.reshape(x, (x.shape[0], -1, self.num_heads, self.depth))
        # (batch_size, seq, num_heads, depth) ->  (batch_size, num_heads, seq, depth)
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])
        return split_x

    def combine_heads(self, x):
        # (batch_size, num_heads, seq, depth) -> (batch_size, seq, num_heads, depth)
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        # (batch_size, seq, num_heads, depth) -> (batch_size, seq, d_model)
        combined_x = tf.reshape(combined_x, (combined_x.shape[0], -1, self.d_model))
        return combined_x

    def call(self, Q, K, V, mask):
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)
        
        # split head
        Q_splits = self.split_heads(WQ)
        K_splits = self.split_heads(WK)
        V_splits = self.split_heads(WV)

        # attention mechanism
        out, attention_weights = self.scaled_dot_product_attention(Q_splits, K_splits, V_splits, mask)
        
        # combine head
        out = self.combine_heads(out)
        out = self.linear(out)

        return out, attention_weights

In [36]:
# Position-wise Feed Forward Network 구현
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        self.fc1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.fc2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        out = self.fc1(x)
        out = self.fc2(out)
            
        return out

In [37]:
# Encoder 레이어 구현
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        ## pre-LN 적용
        # Multi-Head Attention
        identity = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.do(out)
        out += identity
        
        # Position-Wise Feed Forward Network
        identity = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.do(out)
        out += identity
        
        return out, enc_attn

In [38]:
# Decoder 레이어 구현
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.dec_enc_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
    
    def call(self, x, enc_out, dec_enc_mask, dec_mask):
        # Masked Multi-Head Attention
        identity = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, dec_mask)
        out = self.do(out)
        out += identity
        
        # Multi-Head Attention
        identity = out
        out = self.norm_2(x)
        out, dec_enc_attn = self.dec_enc_attn(out, enc_out, enc_out, dec_enc_mask)
        out = self.do(out)
        out += identity
        
        # Position-Wise Feed Forward Network
        identity = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.do(out)
        out += identity

        return out, dec_attn, dec_enc_attn

In [39]:
# Encoder 구현
class Encoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) 
                        for _ in range(n_layers)]
        
    def call(self, x, mask):
        out = x
        enc_attns = []
        for enc_layer in self.enc_layers:
            out, enc_attn = enc_layer(out, mask)
            enc_attns.append(enc_attn)
        return out, enc_attns

In [40]:
# Decoder 구현
class Decoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) 
                            for _ in range(n_layers)]
                            
    def call(self, x, enc_out, dec_enc_mask, dec_mask):
        out = x
    
        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = self.dec_layers[i](out, enc_out, dec_enc_mask, dec_mask)

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns

In [41]:
# Transformer 전체 모델 조립
class Transformer(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    src_vocab_size,
                    tgt_vocab_size,
                    pos_len,
                    dropout=0.2,
                    shared_fc=True,
                    shared_emb=False):
        super(Transformer, self).__init__()
        
        self.d_model = tf.cast(d_model, tf.float32)

        if shared_emb:
            self.enc_emb = self.dec_emb = \
            tf.keras.layers.Embedding(src_vocab_size, d_model)
        else:
            self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
            self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.do = tf.keras.layers.Dropout(dropout)

        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        self.fc = tf.keras.layers.Dense(tgt_vocab_size)

        self.shared_fc = shared_fc

        if shared_fc:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))

    def embedding(self, emb, x):
        seq_len = x.shape[1]
        out = emb(x)

        if self.shared_fc: 
            out *= tf.math.sqrt(self.d_model)

        # pos_encoding (max_seq_len, max_seq_len) vs embedding vector (batch_size, seq_len, d_model)
        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.do(out)

        return out

    def call(self, enc_in, dec_in, enc_mask, dec_enc_mask, dec_mask):
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)

        enc_out, enc_attns = self.encoder(enc_in, enc_mask)
        dec_out, dec_attns, dec_enc_attns = self.decoder(dec_in, enc_out, dec_enc_mask, dec_mask)

        logits = self.fc(dec_out)

        return logits, enc_attns, dec_attns, dec_enc_attns

In [42]:
# 모델 인스턴스 생성
d_model = 256
VOCAB_SIZE = len(tokenizer.word_index)
transformer = Transformer(
    n_layers=2,
    d_model=d_model,
    n_heads=8,
    d_ff=1024,
    src_vocab_size=VOCAB_SIZE,
    tgt_vocab_size=VOCAB_SIZE,
    pos_len=100,
    dropout=0.3,
    shared_fc=True,
    shared_emb=True)

In [43]:
# Learning Rate Scheduler
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()
        
        self.d_model = d_model
        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)
        
        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)

In [44]:
# Learning Rate & Optimizer
learning_rate = LearningRateScheduler(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                        beta_1=0.9,
                                        beta_2=0.98, 
                                        epsilon=1e-9)

In [45]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_) / tf.reduce_sum(mask)

In [46]:
@tf.function()
def train_step(src, tgt, model, optimizer):
    tgt_in = tgt[:, :-1]  # Decoder의 input
    gold = tgt[:, 1:]     # Decoder의 output과 비교하기 위해 right shift를 통해 생성한 최종 타겟

    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, enc_attns, dec_attns, dec_enc_attns

In [47]:
BATCH_SIZE = 64
train_dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).batch(batch_size=BATCH_SIZE)

In [48]:
def evaluate(sentence, model, tokenizer): 
    # start, end = 3, 4
    sentence = preprocess_sentence(sentence)

    tokens = tokenizer.texts_to_sequences([sentence])

    _input = tf.keras.preprocessing.sequence.pad_sequences(tokens,
                                                           maxlen=enc_train.shape[-1],
                                                           padding='post')
    
    ids = []
    output = tf.expand_dims([3], 0)
    for i in range(dec_train.shape[-1]):
        enc_padding_mask, combined_mask, dec_padding_mask = \
        generate_masks(_input, output)

        predictions, enc_attns, dec_attns, dec_enc_attns =\
        model(_input, 
              output,
              enc_padding_mask,
              combined_mask,
              dec_padding_mask)

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()

        if 4 == predicted_id:
            result = tokenizer.sequences_to_texts([ids])
            return result[0], enc_attns, dec_attns, dec_enc_attns

        ids.append(predicted_id)
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

    result = tokenizer.sequences_to_texts([ids])

    return result[0], enc_attns, dec_attns, dec_enc_attns

In [49]:
def generate_answer():
    question_list = ['지루하다, 놀러가고 싶어.', '오늘 일찍 일어났더니 피곤하다.', '간만에 여자친구랑 데이트 하기로 했어.', '집에 있는다는 소리야.']

    for question in question_list:
        print(f"Q: {question}\nA: {evaluate(question, transformer, tokenizer)[0]}\n")
generate_answer()

Q: 지루하다, 놀러가고 싶어.
A: 훅 훅 훅 훅 훅 훅 훅 훅 훅 훅 훅 훅 훅 페친 훅 페친 훅 페친 훅 페친 훅

Q: 오늘 일찍 일어났더니 피곤하다.
A: 훅 페친 훅 훅 훅 훅 훅 훅 훅 훅 훅 훅 페친 페친 페친 페친 페친 페친 페친 길 페친

Q: 간만에 여자친구랑 데이트 하기로 했어.
A: 빠를수록 빠를수록 무시 훅 훅 훅 훅 훅 훅 훅 훅 훅 훅 훅 페친 무시 무시 무시 길 무시 무시

Q: 집에 있는다는 소리야.
A: 훅 훅 훅 훅 훅 훅 훅 훅 훅 훅 훅 훅 훅 페친 훅 페친 훅 페친 훅 페친 훅



In [50]:
EPOCHS = 40
for epoch in range(EPOCHS):
    total_loss = 0
    
    dataset_count = tf.data.experimental.cardinality(train_dataset).numpy()
    tqdm_bar = tqdm(total=dataset_count)
    for src, tgt in train_dataset:
        loss, enc_attns, dec_attns, dec_enc_attnss = train_step(src, tgt, transformer, optimizer)
        total_loss += loss
        tqdm_bar.update(1)
    print(f"epoch {epoch + 1} train loss: {(total_loss / dataset_count):.4f}")
    generate_answer()
    tqdm_bar.close()

  0%|          | 0/353 [00:00<?, ?it/s]

epoch 1 train loss: 5.9298
Q: 지루하다, 놀러가고 싶어.
A: <UNK> 이 게 좋 거 예요 .

Q: 오늘 일찍 일어났더니 피곤하다.
A: 사랑 이 좋 을 거 예요 .

Q: 간만에 여자친구랑 데이트 하기로 했어.
A: <UNK> 이 게 이 네요 .

Q: 집에 있는다는 소리야.
A: <UNK> 이 게 좋 거 예요 .



  0%|          | 0/353 [00:00<?, ?it/s]

epoch 2 train loss: 3.8509
Q: 지루하다, 놀러가고 싶어.
A: <UNK> 를 해 보 세요 .

Q: 오늘 일찍 일어났더니 피곤하다.
A: 좋 은 사람 이 네요 .

Q: 간만에 여자친구랑 데이트 하기로 했어.
A: <UNK> 를 할 수 있 어요 .

Q: 집에 있는다는 소리야.
A: <UNK> 를 해 보 세요 .



  0%|          | 0/353 [00:00<?, ?it/s]

...

epoch 38 train loss: 0.0775
Q: 지루하다, 놀러가고 싶어.
A: 아프 지 말로 아요 .

Q: 오늘 일찍 일어났더니 피곤하다.
A: 그래도 미련 만 먹 는 것 도 중요 해요 .

Q: 간만에 여자친구랑 데이트 하기로 했어.
A: 오류 해 보 세요 .

Q: 집에 있는다는 소리야.
A: 아프 지 말로 아요 .



  0%|          | 0/353 [00:00<?, ?it/s]

epoch 39 train loss: 0.0727
Q: 지루하다, 놀러가고 싶어.
A: 어떤 사랑 이야기 도 그녀 에 대한 제목 은 지 을 수 없 는 것 같 아요 .

Q: 오늘 일찍 일어났더니 피곤하다.
A: 오늘 도 미련 만큼 감사 한 설정 을 이 ㄴ가요 .

Q: 간만에 여자친구랑 데이트 하기로 했어.
A: 도서관 데이트 나 봐요 .

Q: 집에 있는다는 소리야.
A: 어떤 사랑 이야기 도 그녀 에 대한 제목 은 지 을 수 없 는 것 같 아요 .



  0%|          | 0/353 [00:00<?, ?it/s]

epoch 40 train loss: 0.0695
Q: 지루하다, 놀러가고 싶어.
A: 아프 지 말로 아요 .

Q: 오늘 일찍 일어났더니 피곤하다.
A: 아침 도 미련 만큼 부럽 네요 .

Q: 간만에 여자친구랑 데이트 하기로 했어.
A: 하여 에 뽀 켜 세요 .

Q: 집에 있는다는 소리야.
A: 아프 지 말로 아요 .



# 회고

1주일 동안 쉬었더니 transformer에 대한 지식이 초기화 되어서 쉽지 않았던것 같다.

데이터가 많지 않았기 때문에 이번에는 Lexical Substitution으로 데이터 증강을 해서 하는 방법을 사용했다.

한국어 워드벡터로 주어진 셋에서 가난이 부유와 제일 유사하다는 결과가 있었다. 굉장히 마음에 걸리는 포인트이다.

훈련결과를 보았을때 "지루하다, 놀러가고 싶어."와 "집에 있는다는 소리야."의 응답결과가 계속 같게 나왔다. 신기하다....

epoch 20회 쯤에서
```
epoch 20 train loss: 0.1652
Q: 지루하다, 놀러가고 싶어.
A: 아프 지 말로 아요 .

Q: 오늘 일찍 일어났더니 피곤하다.
A: 오늘 일찍 주무세요 .

Q: 간만에 여자친구랑 데이트 하기로 했어.
A: 도서관 엄청 행복 해 주 세요 .

Q: 집에 있는다는 소리야.
A: 아프 지 말로 아요 .
```
정도로 그나마 제일 나아보이는 결과가 있었다. 그 이후에는 과적합 되었는지 더 좋아지지는 않은것 같다.

데이터 증강할때 조사를 좀더 신경써서 잘 제거해주고 했으면 더 결과가 좋지 않았을까 싶다.